### Value at Risk

In [1]:
import scipy.stats
import numpy as np

In [2]:
#!sigma == daily volatility

def VaR(position,sigma,quantile):
    return position * sigma * scipy.stats.norm.ppf(quantile)

In [3]:
scipy.stats.norm.ppf(0.95)

1.6448536269514722

In [4]:
#Interpretation: With an open position of $1000, and a daily volatility of 5%, there is a 95% prob that outcome will not go below
# nor exceed $82
VaR(1000,0.05,0.95)

82.24268134757361

In [5]:
# consider n assets with n weights 

w1 = 0.5
w2 = 0.5
sigma1 = 0.07
sigma2 = 0.03
corr = 0.4


In [6]:
#portfolio variance 
portfoliovar = w1**2 * sigma1**2 + w2**2 * sigma2**2 + 2 * (w1*w2*sigma1*sigma2*corr)

In [7]:
#portfolio volatility 
portfoliovol = portfoliovar**(1/2)

In [8]:
portfoliovol

0.043243496620879306

In [9]:
VaR(1000,portfoliovol,0.95)

71.12922225891705

In [11]:
VaR1 = VaR(1000*w1,sigma1,0.95)
VaR2 = VaR(1000*w2,sigma2,0.95)

In [12]:
vector = np.array([VaR1,VaR2])
vector

array([57.56987694, 24.6728044 ])

In [13]:
corrmatrix = np.array([[1,corr],[corr,1]])
corrmatrix

array([[1. , 0.4],
       [0.4, 1. ]])

In [14]:
(np.dot(np.dot(vector,corrmatrix),vector))**(1/2)

71.12922225891707

In [15]:
import pandas_datareader.data as reader
import datetime as dt

In [16]:
end = dt.datetime.now()
start = dt.datetime(end.year -1, end.month, end.day)

In [17]:
reader.get_data_yahoo(['AAPL','MSFT','TSLA'],start, end)['Adj Close']

Symbols,AAPL,MSFT,TSLA
Date,,,
2021-03-08,115.672493,225.552765,563.000000
2021-03-09,120.374550,231.891144,673.580017
2021-03-10,119.271111,230.542130,668.059998
2021-03-11,121.239410,235.214081,699.599976
2021-03-12,120.314903,233.845230,693.729980
...,...,...,...
2022-03-02,166.559998,300.190002,879.890015
2022-03-03,166.229996,295.920013,839.289978
2022-03-04,163.169998,289.859985,838.289978


In [18]:
df = reader.get_data_yahoo(['AAPL','MSFT','TSLA'],start, end)['Adj Close']

In [19]:
returns = np.log(1+ df.pct_change())
returns

Symbols,AAPL,MSFT,TSLA
Date,,,
2021-03-08,NaN,NaN,NaN
2021-03-09,0.039845,0.027714,0.179327
2021-03-10,-0.009209,-0.005834,-0.008229
2021-03-11,0.016368,0.020062,0.046131
2021-03-12,-0.007655,-0.005837,-0.008426
...,...,...,...
2022-03-02,0.020379,0.017610,0.017796
2022-03-03,-0.001983,-0.014326,-0.047241
2022-03-04,-0.018580,-0.020691,-0.001192


In [20]:
returns.std()

Symbols
AAPL    0.015043
MSFT    0.014452
TSLA    0.035207
dtype: float64

In [21]:
position = df.iloc[-1]
position

Symbols
AAPL    157.440002
MSFT    275.850006
TSLA    824.400024
Name: 2022-03-08 00:00:00, dtype: float64

In [22]:
VaRarray = []

for i in range(len(position)):
    VaRarray.append(VaR(position[i],returns.std()[i],0.95))

In [23]:
VaRarray

[3.895728000272369, 6.557572945973437, 47.74167560689947]

In [24]:
vector = np.array(VaRarray)
vector

array([ 3.895728  ,  6.55757295, 47.74167561])

In [25]:
returns.corr()

Symbols,AAPL,MSFT,TSLA
Symbols,,,
AAPL,1.000000,0.704907,0.449524
MSFT,0.704907,1.000000,0.422167
TSLA,0.449524,0.422167,1.000000


In [26]:
np.dot(np.dot(vector, returns.corr()),vector)**(1/2)

52.96234535952286